In [48]:
import pandas as pd
import numpy as np
import os

Reverse engineering antique_train_queries

In [49]:
full_data_dir = os.path.abspath(os.path.join(os.getcwd(), "..", "..", "..", "data"))
full_file_path = os.path.join(full_data_dir, "antique_test_queries.csv")
lab_file_path = os.path.join(full_data_dir, "antique_train_queries.csv")

full_dataset = pd.read_csv(full_file_path)
lab_dataset = pd.read_csv(lab_file_path)

full_query_ids = set(full_dataset["query_id"].astype(str))
lab_query_ids = set(lab_dataset["query_id"].astype(str))

matching_query_ids = full_query_ids.intersection(lab_query_ids)

print(f"Total unique query_ids in full dataset: {len(full_query_ids)}")
print(f"Total unique query_ids in lab dataset: {len(lab_query_ids)}")
print(f"Number of matching query_ids: {len(matching_query_ids)}")

Total unique query_ids in full dataset: 200
Total unique query_ids in lab dataset: 50
Number of matching query_ids: 50


In [50]:
full_dataset.head()

,query_id,text
0,3990512,how can we get concentration onsomething?
1,714612,Why doesn't the water fall off earth if it's ...
2,2528767,How do I determine the charge of the iron ion ...
3,821387,I have mice.How do I get rid of them humanely?
4,1880028,"What does ""see Leaflet"" mean on Ept Pregnancy ..."


In [51]:
lab_dataset.head()

,query_id,query_embeddings,doc_id,relevance
0,1964316,"[ 8.20141565e-03, 1.82990301e-02, 1.74273867e-...",1964316_5,4
1,1964316,"[ 8.20141565e-03, 1.82990301e-02, 1.74273867e-...",1674088_11,1
2,1964316,"[ 8.20141565e-03, 1.82990301e-02, 1.74273867e-...",1218838_13,2
3,1964316,"[ 8.20141565e-03, 1.82990301e-02, 1.74273867e-...",1519022_15,2
4,1964316,"[ 8.20141565e-03, 1.82990301e-02, 1.74273867e-...",3059341_5,2


In [52]:
len(lab_dataset)

1651

In [53]:
intersection_dataset = full_dataset.merge(lab_dataset, on="query_id")

intersection_dataset.head()

,query_id,text,query_embeddings,doc_id,relevance
0,1880028,"What does ""see Leaflet"" mean on Ept Pregnancy ...","[-2.14085039e-02,-1.47240222e-01,-4.38942723e-...",1880028_3,4
1,1880028,"What does ""see Leaflet"" mean on Ept Pregnancy ...","[-2.14085039e-02,-1.47240222e-01,-4.38942723e-...",1880028_2,2
2,1880028,"What does ""see Leaflet"" mean on Ept Pregnancy ...","[-2.14085039e-02,-1.47240222e-01,-4.38942723e-...",1880028_1,1
3,1880028,"What does ""see Leaflet"" mean on Ept Pregnancy ...","[-2.14085039e-02,-1.47240222e-01,-4.38942723e-...",1880028_0,4
4,1880028,"What does ""see Leaflet"" mean on Ept Pregnancy ...","[-2.14085039e-02,-1.47240222e-01,-4.38942723e-...",845140_9,2


In [54]:
len(intersection_dataset)

1651

In [55]:
test_docs_path = os.path.join(full_data_dir, "antique_test_docs.csv")

test_docs_dataset = pd.read_csv(test_docs_path)

test_doc_ids = set(test_docs_dataset["doc_id"].astype(str))
lab_doc_ids = set(lab_dataset["doc_id"].astype(str))

matching_doc_ids = test_doc_ids.intersection(lab_doc_ids)

print(f"Total unique doc_ids in test dataset: {len(test_doc_ids)}")
print(f"Total unique doc_ids in lab dataset: {len(lab_doc_ids)}")
print(f"Number of matching doc_ids: {len(matching_doc_ids)}")

Total unique doc_ids in test dataset: 403666
Total unique doc_ids in lab dataset: 1649
Number of matching doc_ids: 1649


In [57]:
doc_id_counts = lab_dataset["doc_id"].value_counts()

duplicate_doc_ids = doc_id_counts[doc_id_counts > 1].index.tolist()

print(f"Duplicate doc_id(s): {duplicate_doc_ids}")

duplicate_entries = lab_dataset[lab_dataset["doc_id"].isin(duplicate_doc_ids)]
duplicate_entries.head()

Duplicate doc_id(s): ['3239329_4', '2785579_2']


,query_id,query_embeddings,doc_id,relevance
373,2785579,"[-1.02301352e-02, 1.10469041e-02,-2.07281318e-...",3239329_4,3
378,2785579,"[-1.02301352e-02, 1.10469041e-02,-2.07281318e-...",2785579_2,4
607,4185501,"[ 3.86847258e-02,-4.01194394e-02,-1.45205026e-...",2785579_2,2
972,3239329,"[ 5.15195075e-03,-4.52668220e-02, 3.51459440e-...",3239329_4,2


In [58]:
test_docs_dataset.head()

,doc_id,text
0,2020338_0,A small group of politicians believed strongly...
1,2020338_1,Because there is a lot of oil in Iraq.
2,2020338_2,It is tempting to say that the US invaded Iraq...
3,2020338_3,I think Yuval is pretty spot on. It's a provin...
4,2874684_0,Call an area apiarist. They should be able to...


In [39]:
query_df = intersection_dataset.merge(test_docs_dataset, on="doc_id", suffixes=("_query", "_doc"))

query_df = query_df[["query_id", "doc_id", "query_embeddings", "text_query", "text_doc", "relevance"]]

query_df.rename(columns={"text_query": "query_text", "text_doc": "doc_text"}, inplace=True)

query_df.head()

,query_id,doc_id,query_embeddings,query_text,doc_text,relevance
0,1880028,1880028_3,"[-2.14085039e-02,-1.47240222e-01,-4.38942723e-...","What does ""see Leaflet"" mean on Ept Pregnancy ...",It means the leaflet that should be enclosed i...,4
1,1880028,1880028_2,"[-2.14085039e-02,-1.47240222e-01,-4.38942723e-...","What does ""see Leaflet"" mean on Ept Pregnancy ...",you didn't get a leaf in yours?,2
2,1880028,1880028_1,"[-2.14085039e-02,-1.47240222e-01,-4.38942723e-...","What does ""see Leaflet"" mean on Ept Pregnancy ...",You stuck it in the wrong hole.,1
3,1880028,1880028_0,"[-2.14085039e-02,-1.47240222e-01,-4.38942723e-...","What does ""see Leaflet"" mean on Ept Pregnancy ...",Leaflet is the info inside the package with th...,4
4,1880028,845140_9,"[-2.14085039e-02,-1.47240222e-01,-4.38942723e-...","What does ""see Leaflet"" mean on Ept Pregnancy ...",Nothing is 100% effective in preventing pregna...,2


In [40]:
len(query_df)

1651

In [59]:
doc_id_counts_query = query_df["doc_id"].value_counts()

duplicate_doc_ids_query = doc_id_counts_query[doc_id_counts_query > 1].index.tolist()

print(f"Duplicate doc_id(s) in query_df: {duplicate_doc_ids_query}")

duplicate_entries_query = query_df[query_df["doc_id"].isin(duplicate_doc_ids_query)]

duplicate_entries_query.head()

Duplicate doc_id(s) in query_df: ['2785579_2', '3239329_4']


,query_id,doc_id,query_embeddings,query_text,doc_text,relevance
522,4185501,2785579_2,"[ 3.86847258e-02,-4.01194394e-02,-1.45205026e-...",why does the sun shine?,"I would be sun shine, to make flowers grow, an...",2
759,3239329,3239329_4,"[ 5.15195075e-03,-4.52668220e-02, 3.51459440e-...",Why do we wee a lot in the cold weather especi...,well i think the reason why we have colder wea...,2
1372,2785579,3239329_4,"[-1.02301352e-02, 1.10469041e-02,-2.07281318e-...","If you could be an element of weather, which w...",well i think the reason why we have colder wea...,3
1377,2785579,2785579_2,"[-1.02301352e-02, 1.10469041e-02,-2.07281318e-...","If you could be an element of weather, which w...","I would be sun shine, to make flowers grow, an...",4


Reverse engineering antique_doc_embeddings

In [42]:
full_data_dir = os.path.abspath(os.path.join(os.getcwd(), "..", "..", "..", "data"))
full_file_path = os.path.join(full_data_dir, "antique_test_docs.csv")
lab_file_path = os.path.join(full_data_dir, "antique_doc_embeddings.csv")

full_dataset = pd.read_csv(full_file_path)
lab_dataset = pd.read_csv(lab_file_path)

full_doc_ids = set(full_dataset["doc_id"].astype(str))
lab_doc_ids = set(lab_dataset["doc_id"].astype(str))

matching_doc_ids = full_doc_ids.intersection(lab_doc_ids)

print(f"Total unique doc_ids in full dataset: {len(full_doc_ids)}")
print(f"Total unique doc_ids in lab dataset: {len(lab_doc_ids)}")
print(f"Number of matching doc_ids: {len(matching_doc_ids)}")

Total unique doc_ids in full dataset: 403666
Total unique doc_ids in lab dataset: 6486
Number of matching doc_ids: 6486


In [44]:
len(lab_dataset)

6486

In [43]:
document_df = full_dataset.merge(lab_dataset, on="doc_id")

document_df.head()

,doc_id,text,doc_embeddings
0,1166941_1,Some people believe that if there is a sick pe...,"[-2.13019364e-02,-7.64960945e-02, 1.42080365e-..."
1,563604_1,See a dentist every 6 months,"[ 4.08016555e-02, 1.52095212e-02, 1.16801914e-..."
2,3593337_1,Get on your parents insurance policy.,"[-3.83477509e-02, 6.78179637e-02,-1.89724788e-..."
3,1814059_0,if u have young kids u want to take them to Kn...,"[-3.37657854e-02, 2.19965018e-02,-4.47764695e-..."
4,2855289_1,The purpose of laboratory testing of seed germ...,"[-5.14386334e-02,-7.02764839e-02, 4.13872185e-..."


In [45]:
len(document_df)

6486